In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from collections import Counter
import os
import time
import re
from glob import glob
import glob as glob_module
import matplotlib.pyplot as plt
import seaborn as sns
import gc

try:
    from thop import profile
    THOP_AVAILABLE = True
except ImportError:
    print("Warning: thop not available. Install with: pip install thop")
    THOP_AVAILABLE = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True

BATCH_SIZE = 512
GRADIENT_ACCUMULATION_STEPS = 2
EPOCHS = 50
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4
NUM_WORKERS = 4

HIDDEN_DIM = 128
LATENT_DIM = 64
NUM_FREQUENCIES = 16
SIREN_OMEGA = 30.0
NUM_SIREN_LAYERS = 3

RANDOM_SAMPLING = True
SAMPLING_RATIO = 0.5
DERIVATIVE_SUPERVISION = False
DOWNSAMPLE_EXPERIMENT = True
DOWNSAMPLE_POINTS = 32

USE_AMP = True

print("="*70)
print("CONFIGURATION")
print("="*70)
print(f"Batch Size: {BATCH_SIZE} (Effective: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS})")
print(f"Gradient Accumulation Steps: {GRADIENT_ACCUMULATION_STEPS}")
print(f"Epochs: {EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Hidden Dim: {HIDDEN_DIM} | Latent Dim: {LATENT_DIM}")
print(f"SIREN Layers: {NUM_SIREN_LAYERS} | Frequencies: {NUM_FREQUENCIES}")
print(f"Mixed Precision: {USE_AMP}")
print(f"Num Workers: {NUM_WORKERS}")
print("="*70)


class UCIHARDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


def read_txt_matrix(file_path):
    return np.loadtxt(file_path)


def load_uci_har(root_path='/content/drive/MyDrive/HAR_Dataset/UCI'):
    print(f"\n[UCI-HAR] Loading from: {root_path}")
    UCI_CHANNELS_PREFIX = [
        "total_acc_x_", "total_acc_y_", "total_acc_z_",
        "body_acc_x_", "body_acc_y_", "body_acc_z_",
        "body_gyro_x_", "body_gyro_y_", "body_gyro_z_",
    ]
    def load_split(split):
        channels = []
        for prefix in UCI_CHANNELS_PREFIX:
            file_path = os.path.join(root_path, f"{prefix}{split}.txt")
            print(f"  Loading: {os.path.basename(file_path)}")
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"File not found: {file_path}")
            channels.append(read_txt_matrix(file_path))
        X = np.stack(channels, axis=1)
        y = read_txt_matrix(os.path.join(root_path, f"y_{split}.txt")).astype(int) - 1
        return X, y

    X_train, y_train = load_split('train')
    X_test, y_test = load_split('test')
    X_train = X_train.transpose(0, 2, 1)
    X_test = X_test.transpose(0, 2, 1)
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    scaler = StandardScaler()
    scaler.fit(X_train_flat)
    X_train_flat = scaler.transform(X_train_flat)
    X_test_flat = scaler.transform(X_test_flat)
    X_train = X_train_flat.reshape(X_train.shape)
    X_test = X_test_flat.reshape(X_test.shape)
    activity_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']
    print(f"  Train: {X_train.shape}, Test: {X_test.shape}")
    return X_train, y_train.astype(np.int64), X_test, y_test.astype(np.int64), activity_names


def load_wisdm_data(dataset_path="/content/drive/MyDrive/HAR_Dataset/WISDM"):
    print(f"\n[WISDM] Loading from: {dataset_path}")
    if os.path.isfile(dataset_path):
        file_paths = [dataset_path]
    else:
        possible_files = ['WISDM_ar_v1.1_raw.txt', 'WISDM_ar_v1.1_trans.arff', 'wisdm-dataset.txt', 'actitracker_raw.txt']
        file_paths = []
        for filename in possible_files:
            full_path = os.path.join(dataset_path, filename)
            if os.path.exists(full_path):
                file_paths.append(full_path)
                print(f"  Found: {filename}")
    if not file_paths:
        return None, None, None, None, None

    all_data = []
    for file_path in file_paths:
        print(f"  Processing: {os.path.basename(file_path)}")
        with open(file_path, 'r') as f:
            lines = f.readlines()
        cleaned_data = []
        for line in lines:
            line = line.strip()
            if not line:
                continue
            line = line.rstrip(';').rstrip(',')
            if ',' in line:
                parts = line.split(',')
            elif ';' in line:
                parts = line.split(';')
            else:
                continue
            if len(parts) < 6:
                continue
            try:
                user = parts[0].strip()
                activity = parts[1].strip()
                timestamp = parts[2].strip()
                x_str = parts[3].strip()
                y_str = parts[4].strip()
                z_str = parts[5].strip()
                if ';' in x_str:
                    x_str = x_str.split(';')[0]
                if ';' in y_str:
                    y_str = y_str.split(';')[0]
                if ';' in z_str:
                    z_str = z_str.split(';')[0]
                x = float(x_str)
                y = float(y_str)
                z = float(z_str)
                cleaned_data.append([user, activity, timestamp, x, y, z])
            except (ValueError, IndexError):
                continue
        if cleaned_data:
            df = pd.DataFrame(cleaned_data, columns=['user', 'activity', 'timestamp', 'x', 'y', 'z'])
            df['x'] = pd.to_numeric(df['x'], errors='coerce')
            df['y'] = pd.to_numeric(df['y'], errors='coerce')
            df['z'] = pd.to_numeric(df['z'], errors='coerce')
            df = df.dropna()
            all_data.append(df)

    if not all_data:
        return None, None, None, None, None

    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.dropna()
    combined_df = combined_df[combined_df['activity'].str.strip() != '']
    all_windows = []
    all_labels = []
    groups = combined_df.groupby(['user', 'activity']) if 'user' in combined_df.columns else combined_df.groupby(['activity'])
    window_size = 80
    step = 40
    for group_name, group_data in groups:
        activity = group_name[-1] if isinstance(group_name, tuple) else group_name
        acc_data = group_data[['x', 'y', 'z']].values.astype(np.float32)
        if len(acc_data) < window_size:
            continue
        start = 0
        while start + window_size <= len(acc_data):
            window_data = acc_data[start:start + window_size, :]
            all_windows.append(window_data)
            all_labels.append(activity)
            start += step

    if not all_windows:
        return None, None, None, None, None

    X_windowed = np.array(all_windows, dtype=np.float32)
    scaler = StandardScaler()
    X_windowed_flat = X_windowed.reshape(X_windowed.shape[0], -1)
    X_windowed_flat = scaler.fit_transform(X_windowed_flat)
    X_windowed = X_windowed_flat.reshape(X_windowed.shape)
    X_windowed = X_windowed.transpose(0, 2, 1)
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(all_labels)
    class_names = [str(label) for label in label_encoder.classes_]
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Train: {X_train.shape}, Test: {X_test.shape}")
    return X_train, y_train, X_test, y_test, class_names


def load_pamap2_data(dataset_dir="/content/drive/MyDrive/HAR_Dataset/PAMAP2"):
    print(f"\n[PAMAP2] Loading from: {dataset_dir}")
    file_paths = sorted(glob(os.path.join(dataset_dir, 'Protocol', 'subject*.dat')))
    optional_path = os.path.join(dataset_dir, 'Optional')
    if os.path.exists(optional_path):
        file_paths += sorted(glob(os.path.join(optional_path, 'subject*.dat')))
    if not file_paths:
        return None, None, None, None, None

    print(f"  Found {len(file_paths)} subject files")

    activity_labels = [
        "lying", "sitting", "standing", "walking", "running", "cycling",
        "Nordic walking", "ascending stairs", "descending stairs",
        "vacuum cleaning", "ironing", "rope jumping"
    ]
    label_to_activity_idx = {
        1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 12: 7, 13: 8, 16: 9, 17: 10, 24: 11
    }
    all_windows = []
    all_labels = []
    window_size = 100
    step = 50
    for file_path in file_paths:
        print(f"  Processing: {os.path.basename(file_path)}")
        try:
            df = pd.read_csv(file_path, sep=r'\s+', header=None, na_values='NaN')
        except:
            continue
        df_cleaned = df.ffill().bfill()
        if df_cleaned.empty:
            continue
        labels = df_cleaned.iloc[:, 1].values.astype(int)
        all_sensor_cols = list(range(4, 10)) + list(range(21, 27)) + list(range(38, 44))
        if df_cleaned.shape[1] < max(all_sensor_cols) + 1:
            continue
        features = df_cleaned.iloc[:, all_sensor_cols].values.astype(np.float32)
        valid_indices = np.where(np.isin(labels, list(label_to_activity_idx.keys())))[0]
        if len(valid_indices) == 0:
            continue
        features = features[valid_indices, :]
        labels = labels[valid_indices]
        if len(features) < window_size:
            continue
        start = 0
        while start + window_size <= len(features):
            window_data = features[start : start + window_size, :]
            window_labels_raw = labels[start : start + window_size]
            most_common_label = Counter(window_labels_raw).most_common(1)[0][0]
            if most_common_label in label_to_activity_idx:
                all_windows.append(window_data)
                all_labels.append(label_to_activity_idx[most_common_label])
            start += step

    if not all_windows:
        return None, None, None, None, None

    X_windowed = np.array(all_windows, dtype=np.float32)
    y_encoded = np.array(all_labels, dtype=int)
    scaler = StandardScaler()
    X_windowed_flat = X_windowed.reshape(X_windowed.shape[0], -1)
    X_windowed_flat = scaler.fit_transform(X_windowed_flat)
    X_windowed = X_windowed_flat.reshape(X_windowed.shape)
    X_windowed = X_windowed.transpose(0, 2, 1)
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Train: {X_train.shape}, Test: {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_labels


def load_mhealth_data(dataset_dir="/content/drive/MyDrive/HAR_Dataset/MHEALTH"):
    print(f"\n[MHEALTH] Loading from: {dataset_dir}")
    if not os.path.exists(dataset_dir):
        return None, None, None, None, None
    subject_files = sorted([
        os.path.join(dataset_dir, f)
        for f in os.listdir(dataset_dir)
        if f.startswith("mHealth_subject") and f.endswith(".log")
    ])
    if not subject_files:
        return None, None, None, None, None

    print(f"  Found {len(subject_files)} subject files")

    all_windows = []
    all_labels = []
    window_size = 50
    step = 25
    for file_path in subject_files:
        print(f"  Processing: {os.path.basename(file_path)}")
        try:
            df = pd.read_csv(file_path, sep=r'\s+', header=None, engine='python', dtype=np.float32)
            df = df.ffill().bfill()
            if df.shape[1] < 24:
                continue
            labels = df.iloc[:, 23].values.astype(int)
            imu_cols = [0, 1, 2] + list(range(5, 23))
            features = df.iloc[:, imu_cols].values
            valid_indices = np.where(labels != 0)[0]
            if len(valid_indices) == 0:
                continue
            features = features[valid_indices, :]
            labels = labels[valid_indices]
            if len(features) < window_size:
                continue
            start = 0
            while start + window_size <= len(features):
                window_data = features[start : start + window_size, :]
                window_labels_raw = labels[start : start + window_size]
                most_common_label = Counter(window_labels_raw).most_common(1)[0][0]
                all_windows.append(window_data)
                all_labels.append(most_common_label)
                start += step
        except:
            continue

    if not all_windows:
        return None, None, None, None, None

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(all_labels)
    X_windowed = np.array(all_windows, dtype=np.float32)
    scaler = StandardScaler()
    X_windowed_flat = X_windowed.reshape(X_windowed.shape[0], -1)
    X_windowed_flat = scaler.fit_transform(X_windowed_flat)
    X_windowed = X_windowed_flat.reshape(X_windowed.shape)
    X_windowed = X_windowed.transpose(0, 2, 1)
    mhealth_activity_mapping = {
        1: 'Standing still', 2: 'Sitting and relaxing', 3: 'Lying down', 4: 'Walking',
        5: 'Climbing stairs', 6: 'Waist bends forward', 7: 'Frontal elevation of arms',
        8: 'Knees bending', 9: 'Cycling', 10: 'Jogging', 11: 'Running', 12: 'Jump front & back'
    }
    activity_labels = []
    class_names = list(label_encoder.classes_)
    for encoded_idx in range(len(class_names)):
        original_label = class_names[encoded_idx]
        activity_labels.append(mhealth_activity_mapping.get(original_label, f"Unknown_Activity_{original_label}"))
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Train: {X_train.shape}, Test: {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_labels


def slide_window(data, w_s, stride):
    windows = []
    start = 0
    while start + w_s <= len(data):
        windows.append(data[start:start + w_s, :])
        start += stride
    if not windows:
        return np.array([])
    return np.array(windows)


def load_mobiact_sensor_file(path):
    data = []
    try:
        with open(path, 'r') as f:
            in_data_section = False
            for line in f:
                if line.startswith('@DATA'):
                    in_data_section = True
                    continue
                if not in_data_section:
                    continue
                parts = line.strip().split(',')
                if len(parts) == 4:
                    try:
                        data.append([float(p) for p in parts])
                    except ValueError:
                        continue
    except Exception as e:
        return None
    if not data:
        return None
    return np.array(data, dtype=np.float32)


def load_mobiact_data(data_path, w_s=100, stride=50):
    print(f"\n[MobiAct] Loading from: {data_path}")
    ADL_CODES = ['STD', 'WAL', 'JOG', 'JUM', 'STU', 'STN', 'SCH', 'CSI', 'CSO']
    ALL_ACTIVITIES = ADL_CODES
    label_encoder = LabelEncoder()
    label_encoder.fit(ALL_ACTIVITIES)
    activity_names = list(label_encoder.classes_)
    subject_dirs = sorted(glob_module.glob(os.path.join(data_path, 'sub*')))
    subjects_to_use = []
    for sub_dir in subject_dirs:
        if os.path.isdir(os.path.join(sub_dir, 'ADL')) and os.path.isdir(os.path.join(sub_dir, 'FALLS')):
            subjects_to_use.append(os.path.basename(sub_dir))

    print(f"  Found {len(subjects_to_use)} subjects")

    all_data_for_scaler = []
    all_windows, all_labels, all_subjects = [], [], []
    temp_data_store = []
    for sub_name in subjects_to_use:
        print(f"  Processing subject: {sub_name}")
        subject_id_match = re.search(r'(\d+)', sub_name)
        if not subject_id_match:
            continue
        subject_id = int(subject_id_match.group(0))
        for act_type_folder in ['ADL']:
            act_folders_path = os.path.join(data_path, sub_name, act_type_folder, '*')
            for act_folder_path in glob_module.glob(act_folders_path):
                act_code = os.path.basename(act_folder_path)
                if act_code not in ALL_ACTIVITIES:
                    continue
                label = label_encoder.transform([act_code])[0]
                trial_files = glob_module.glob(os.path.join(act_folder_path, f'{act_code}_acc_{subject_id}_*.txt'))
                trials = sorted(list(set([f.split('_')[-1].split('.')[0] for f in trial_files])))
                for trial in trials:
                    acc_file = os.path.join(act_folder_path, f'{act_code}_acc_{subject_id}_{trial}.txt')
                    gyro_file = os.path.join(act_folder_path, f'{act_code}_gyro_{subject_id}_{trial}.txt')
                    ori_file = os.path.join(act_folder_path, f'{act_code}_ori_{subject_id}_{trial}.txt')
                    if not all(os.path.exists(f) for f in [acc_file, gyro_file, ori_file]):
                        continue
                    data_acc = load_mobiact_sensor_file(acc_file)
                    data_gyro = load_mobiact_sensor_file(gyro_file)
                    data_ori = load_mobiact_sensor_file(ori_file)
                    if any(d is None for d in [data_acc, data_gyro, data_ori]):
                        continue
                    L = min(len(data_acc), len(data_gyro), len(data_ori))
                    if L < w_s:
                        continue
                    combined_data = np.hstack((data_acc[:L, 1:], data_gyro[:L, 1:], data_ori[:L, 1:]))
                    combined_data = combined_data.astype(np.float32)
                    combined_data = pd.DataFrame(combined_data).ffill().bfill().values
                    if np.isnan(combined_data).any():
                        continue
                    all_data_for_scaler.append(combined_data)
                    temp_data_store.append({
                        'data': combined_data,
                        'label': label,
                        'subject': subject_id
                    })

    if not all_data_for_scaler:
        return None, None, None, None, None

    scaler = StandardScaler().fit(np.vstack(all_data_for_scaler))
    for item in temp_data_store:
        scaled_data = scaler.transform(item['data'])
        scaled_data = np.nan_to_num(scaled_data)
        windows = slide_window(scaled_data, w_s, stride)
        if windows.shape[0] > 0:
            all_windows.append(windows)
            all_labels.extend([item['label']] * windows.shape[0])
            all_subjects.extend([item['subject']] * windows.shape[0])

    if not all_windows:
        return None, None, None, None, None

    X_windowed = np.vstack(all_windows).astype(np.float32)
    X_windowed = np.transpose(X_windowed, (0, 2, 1))
    y_encoded = np.array(all_labels, dtype=int)
    X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42)
    print(f"  Train: {X_train.shape}, Test: {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_names


class MotionSenseLoader:
    def __init__(self, frame_len, feature_name, N_classes):
        self.feature_names = feature_name
        self.N_Feature = len(feature_name)
        self.frame_length = frame_len
        self.hop_size = frame_len//2
        self.N_classes = N_classes
        self.label_encoder = OneHotEncoder(sparse_output=False)

    def framing(self, signal):
        shape = ((signal.shape[0] - self.frame_length) // self.hop_size + 1, self.frame_length)
        strides = (signal.strides[0] * self.hop_size, signal.strides[0])
        return np.lib.stride_tricks.as_strided(signal, shape=shape, strides=strides)

    def create_label(self, label):
        return self.label_encoder.fit_transform(label)

    def load_trainings_data(self, files, label_frame):
        label = self.create_label(label_frame)
        self.trainings_data, self.trainings_label = self.load_data(files, label)

    def load_validation_data(self, files, label_frame):
        label = self.label_encoder.transform(label_frame)
        self.validation_data, self.validation_label = self.load_data(files, label)

    def load_data(self, files, label):
        feature_matrix, label_matrix = None, None
        for i in range(len(files)):
            try:
                tmp_data = pd.read_csv(files[i], engine='python')
            except:
                continue
            N_Blocks = 1+(np.shape(tmp_data)[0]-self.frame_length)//self.hop_size
            if N_Blocks <= 0:
                continue
            tmp_feature_mat = np.zeros((N_Blocks, self.frame_length, self.N_Feature))
            tmp_label_vec = np.zeros((N_Blocks, self.N_classes))
            for j in range(N_Blocks):
                tmp_label_vec[j, :] = label[i, :]
            for idf, feat in enumerate(self.feature_names):
                frame_matrix = self.framing(tmp_data[feat].to_numpy())
                tmp_feature_mat[:, :, idf] = frame_matrix[:N_Blocks]
            if feature_matrix is None:
                feature_matrix = tmp_feature_mat
                label_matrix = tmp_label_vec
            else:
                feature_matrix = np.append(feature_matrix, tmp_feature_mat, axis=0)
                label_matrix = np.append(label_matrix, tmp_label_vec, axis=0)
        return feature_matrix, label_matrix


def load_motionsense_data(root_path='/content/drive/MyDrive/HAR_Dataset/MOTIONSENSE'):
    print(f"\n[MotionSense] Loading from: {root_path}")
    files, label = [], []
    for dirname, _, filenames in os.walk(root_path):
        for filename in filenames:
            if filename.endswith('.csv') and not filename.startswith('.'):
                full_path = os.path.join(dirname, filename)
                if 'sub_' in filename:
                    files.append(full_path)
                    parent_dir = os.path.basename(os.path.dirname(full_path))
                    if '_' in parent_dir:
                        label.append(parent_dir.split('_')[0])
                    else:
                        files.pop()
    if not files:
        return None, None, None, None, None

    print(f"  Found {len(files)} CSV files")

    label_frame = pd.DataFrame(label, columns=['act'])
    files_train, files_valid, y_train_raw, y_valid_raw = train_test_split(files, label_frame, test_size=0.2, random_state=0)
    Feature = ['attitude.roll','attitude.pitch','attitude.yaw','gravity.x','gravity.y','gravity.z',
               'rotationRate.x','rotationRate.y','rotationRate.z','userAcceleration.x','userAcceleration.y','userAcceleration.z']
    N_classes = 6
    loader = MotionSenseLoader(128, Feature, N_classes)
    print(f"  Processing training files...")
    loader.load_trainings_data(files_train, y_train_raw)
    print(f"  Processing validation files...")
    loader.load_validation_data(files_valid, y_valid_raw)
    X_train = loader.trainings_data.astype(np.float32).transpose(0, 2, 1)
    X_test = loader.validation_data.astype(np.float32).transpose(0, 2, 1)
    y_train = np.argmax(loader.trainings_label, axis=1)
    y_test = np.argmax(loader.validation_label, axis=1)
    activity_names = list(loader.label_encoder.categories_[0])
    print(f"  Train: {X_train.shape}, Test: {X_test.shape}")
    return X_train, y_train, X_test, y_test, activity_names


class FourierFeatureMapping(nn.Module):
    def __init__(self, num_frequencies, scale=1.0):
        super().__init__()
        self.num_frequencies = num_frequencies
        self.scale = scale
        self.B = nn.Parameter(
            torch.randn(1, num_frequencies) * scale,
            requires_grad=False
        )

    def forward(self, x):
        x_proj = 2 * np.pi * x @ self.B
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)


class SirenLayer(nn.Module):
    def __init__(self, in_features, out_features, omega=30.0, is_first=False):
        super().__init__()
        self.omega = omega
        self.is_first = is_first
        self.linear = nn.Linear(in_features, out_features)
        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.linear.in_features,
                                             1 / self.linear.in_features)
            else:
                self.linear.weight.uniform_(
                    -np.sqrt(6 / self.linear.in_features) / self.omega,
                    np.sqrt(6 / self.linear.in_features) / self.omega
                )

    def forward(self, x):
        return torch.sin(self.omega * self.linear(x))


class ImplicitMotionField(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, num_layers=3):
        super().__init__()
        self.latent_dim = latent_dim
        self.fourier_mapping = FourierFeatureMapping(NUM_FREQUENCIES, scale=10.0)

        input_dim = latent_dim + 2 * NUM_FREQUENCIES

        self.siren1 = SirenLayer(input_dim, hidden_dim, SIREN_OMEGA, is_first=True)
        self.siren2 = SirenLayer(hidden_dim, hidden_dim, SIREN_OMEGA)
        self.siren3 = SirenLayer(hidden_dim, hidden_dim, SIREN_OMEGA)

        self.final_layer = nn.Linear(hidden_dim, output_dim)

        with torch.no_grad():
            self.final_layer.weight.uniform_(
                -np.sqrt(6 / hidden_dim) / SIREN_OMEGA,
                np.sqrt(6 / hidden_dim) / SIREN_OMEGA
            )

    def forward(self, latent_code, time_coords):
        batch_size, num_samples, _ = time_coords.shape
        latent_expanded = latent_code.unsqueeze(1).expand(-1, num_samples, -1)
        time_features = self.fourier_mapping(time_coords.reshape(-1, 1))
        time_features = time_features.reshape(batch_size, num_samples, -1)

        x = torch.cat([latent_expanded, time_features], dim=-1)
        x = x.reshape(-1, x.shape[-1])

        x = self.siren1(x)
        x = self.siren2(x)
        x = self.siren3(x)
        x = self.final_layer(x)

        x = x.reshape(batch_size, num_samples, -1)
        return x


class LightweightEncoder(nn.Module):
    def __init__(self, input_channels, latent_dim):
        super().__init__()

        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(128)

        self.pool = nn.MaxPool1d(2)
        self.gap = nn.AdaptiveAvgPool1d(1)

        self.fc_latent = nn.Linear(128, latent_dim)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)

        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)

        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)

        x = self.gap(x)
        x = x.squeeze(-1)

        latent = self.fc_latent(x)
        return latent


class MotionStatisticsExtractor(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        self.input_channels = input_channels

    def forward(self, motion_field):
        mean = motion_field.mean(dim=2)
        std = motion_field.std(dim=2)
        min_val = motion_field.min(dim=2)[0]
        max_val = motion_field.max(dim=2)[0]

        with autocast('cuda', enabled=False):
            motion_field_fp32 = motion_field.float()
            fft = torch.fft.rfft(motion_field_fp32, dim=2)
            fft_mag = torch.abs(fft)
            fft_features = fft_mag[:, :, :5].flatten(1)

        features = torch.cat([mean, std, min_val, max_val, fft_features], dim=1)
        return features


class LightweightIMF(nn.Module):
    def __init__(self, input_channels, hidden_dim, latent_dim, num_classes, seq_length):
        super().__init__()
        self.seq_length = seq_length
        self.input_channels = input_channels

        self.encoder = LightweightEncoder(input_channels, latent_dim)

        self.motion_field = ImplicitMotionField(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            output_dim=input_channels,
            num_layers=3
        )

        self.stat_extractor = MotionStatisticsExtractor(input_channels)

        stat_dim = input_channels * 4 + input_channels * 5

        self.classifier = nn.Sequential(
            nn.Linear(latent_dim + stat_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x, sample_times=None, num_samples=None, return_latent=False):
        batch_size = x.shape[0]
        latent = self.encoder(x)

        if sample_times is None:
            if num_samples is None:
                num_samples = self.seq_length
            sample_times = torch.linspace(0, 1, num_samples, device=x.device)
            sample_times = sample_times.unsqueeze(0).unsqueeze(-1)
            sample_times = sample_times.expand(batch_size, -1, -1)

        motion_recon = self.motion_field(latent, sample_times)
        motion_recon = motion_recon.permute(0, 2, 1)

        motion_stats = self.stat_extractor(motion_recon)
        combined_features = torch.cat([latent, motion_stats], dim=1)
        logits = self.classifier(combined_features)

        if return_latent:
            return logits, motion_recon, latent
        return logits, motion_recon, latent


def compute_flops_params(model, input_shape, device):
    if not THOP_AVAILABLE:
        return 0.0, 0.0

    model.eval()
    dummy_input = torch.randn(1, *input_shape).to(device)
    macs, params = profile(model, inputs=(dummy_input,), verbose=False)
    flops_m = macs * 2 / 1e6
    params_m = params / 1e6
    return flops_m, params_m


def measure_inference_time(model, input_shape, device, n_runs=100, warmup=10):
    model.eval()
    dummy_input = torch.randn(1, *input_shape).to(device)

    with torch.no_grad():
        for _ in range(warmup):
            _ = model(dummy_input)

    if device.type == 'cuda':
        torch.cuda.synchronize()

    start = time.time()
    with torch.no_grad():
        for _ in range(n_runs):
            _ = model(dummy_input)

    if device.type == 'cuda':
        torch.cuda.synchronize()

    end = time.time()
    return (end - start) / n_runs * 1000


def compute_numerical_derivatives(signal, dt=1.0):
    velocity = torch.zeros_like(signal)
    velocity[:, :, 1:-1] = (signal[:, :, 2:] - signal[:, :, :-2]) / (2 * dt)
    velocity[:, :, 0] = (signal[:, :, 1] - signal[:, :, 0]) / dt
    velocity[:, :, -1] = (signal[:, :, -1] - signal[:, :, -2]) / dt
    return velocity


def plot_confusion_matrix(cm, class_names, dataset_name):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {dataset_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{dataset_name}.png', dpi=150)
    plt.close()


def plot_tsne(latents, labels, class_names, dataset_name):
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    latents_2d = tsne.fit_transform(latents)

    plt.figure(figsize=(12, 10))
    for i, class_name in enumerate(class_names):
        mask = labels == i
        plt.scatter(latents_2d[mask, 0], latents_2d[mask, 1],
                   label=class_name, alpha=0.6, s=20)
    plt.legend()
    plt.title(f't-SNE Visualization - {dataset_name}')
    plt.xlabel('t-SNE 1')
    plt.ylabel('t-SNE 2')
    plt.tight_layout()
    plt.savefig(f'tsne_{dataset_name}.png', dpi=150)
    plt.close()


def train_epoch(model, train_loader, optimizer, scaler, epoch, seq_length):
    model.train()
    total_loss = 0
    total_ce = 0
    total_recon = 0
    total_vel = 0
    total_smooth = 0
    correct = 0
    total = 0

    optimizer.zero_grad()

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        batch_size = data.shape[0]

        with autocast('cuda', enabled=USE_AMP):
            if RANDOM_SAMPLING and epoch > 5:
                num_observed = int(seq_length * SAMPLING_RATIO)
                observed_indices = torch.randperm(seq_length)[:num_observed]
                observed_indices, _ = torch.sort(observed_indices)

                observed_times = observed_indices.float() / (seq_length - 1)
                observed_times = observed_times.unsqueeze(0).unsqueeze(-1).expand(batch_size, -1, -1).to(device)
                observed_data = data[:, :, observed_indices]

                latent = model.encoder(data)
                motion_observed = model.motion_field(latent, observed_times)
                motion_observed = motion_observed.permute(0, 2, 1)

                full_times = torch.linspace(0, 1, seq_length, device=device)
                full_times = full_times.unsqueeze(0).unsqueeze(-1).expand(batch_size, -1, -1)
                motion_full = model.motion_field(latent, full_times)
                motion_full = motion_full.permute(0, 2, 1)

                motion_stats = model.stat_extractor(motion_full)
                combined_features = torch.cat([latent, motion_stats], dim=1)
                logits = model.classifier(combined_features)

                recon_loss = nn.functional.mse_loss(motion_observed, observed_data)
            else:
                logits, motion_full, latent = model(data)
                recon_loss = nn.functional.mse_loss(motion_full, data)

            ce_loss = nn.functional.cross_entropy(logits, target)

            if DERIVATIVE_SUPERVISION:
                num_deriv_points = 64
                deriv_times = torch.linspace(0, 1, num_deriv_points, device=device)
                deriv_times = deriv_times.unsqueeze(0).unsqueeze(-1).expand(batch_size, -1, -1)

                with autocast('cuda', enabled=False):
                    deriv_times_grad = deriv_times.clone().requires_grad_(True)
                    motion_at_times = model.motion_field(latent.float(), deriv_times_grad)

                    velocity_implicit = []
                    for i in range(motion_at_times.shape[2]):
                        grad_outputs = torch.ones_like(motion_at_times[:, :, i])
                        grad = torch.autograd.grad(
                            outputs=motion_at_times[:, :, i],
                            inputs=deriv_times_grad,
                            grad_outputs=grad_outputs,
                            create_graph=True,
                            retain_graph=True
                        )[0]
                        velocity_implicit.append(grad.squeeze(-1))

                    velocity_implicit = torch.stack(velocity_implicit, dim=2)

                    dt_full = 1.0 / (seq_length - 1)
                    velocity_gt_full = compute_numerical_derivatives(data.float(), dt=dt_full)

                    deriv_indices_full = (deriv_times.squeeze(-1) * (seq_length - 1)).clamp(0, seq_length - 1).long()
                    velocity_numerical_list = []
                    for i in range(batch_size):
                        velocity_numerical_list.append(velocity_gt_full[i, :, deriv_indices_full[i]])

                    velocity_numerical = torch.stack(velocity_numerical_list, dim=0)
                    velocity_numerical = velocity_numerical.permute(0, 2, 1)

                    velocity_loss = nn.functional.l1_loss(velocity_implicit, velocity_numerical)
            else:
                velocity_loss = torch.tensor(0.0, device=device)

            random_times = torch.rand(batch_size, 50, 1, device=device)
            dt_smoothness = 0.01
            t1 = torch.clamp(random_times, 0, 1 - dt_smoothness)
            t2 = t1 + dt_smoothness
            m1 = model.motion_field(latent, t1)
            m2 = model.motion_field(latent, t2)
            temporal_derivative = (m2 - m1) / dt_smoothness
            smoothness_loss = torch.mean(temporal_derivative ** 2)

            loss = ce_loss + 0.5 * recon_loss + 0.1 * velocity_loss + 0.01 * smoothness_loss
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        scaler.scale(loss).backward()

        if (batch_idx + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS
        total_ce += ce_loss.item()
        total_recon += recon_loss.item()
        total_vel += velocity_loss.item()
        total_smooth += smoothness_loss.item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)

    return {
        'loss': total_loss / len(train_loader),
        'ce': total_ce / len(train_loader),
        'recon': total_recon / len(train_loader),
        'vel': total_vel / len(train_loader),
        'smooth': total_smooth / len(train_loader),
        'acc': 100. * correct / total
    }


def evaluate(model, test_loader, return_latents=False):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    all_latents = []

    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)

            with autocast('cuda', enabled=USE_AMP):
                logits, motion_recon, latent = model(data, return_latent=True)

                ce_loss = nn.functional.cross_entropy(logits, target)
                recon_loss = nn.functional.mse_loss(motion_recon, data)
                loss = ce_loss + 0.5 * recon_loss

            total_loss += loss.item()

            pred = logits.argmax(dim=1)
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

            if return_latents:
                all_latents.append(latent.cpu().numpy())

    avg_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_targets, all_preds) * 100
    f1 = f1_score(all_targets, all_preds, average='weighted') * 100
    precision = precision_score(all_targets, all_preds, average='weighted', zero_division=0) * 100
    recall = recall_score(all_targets, all_preds, average='weighted', zero_division=0) * 100

    if return_latents:
        all_latents = np.vstack(all_latents)
        return avg_loss, accuracy, f1, precision, recall, all_preds, all_targets, all_latents

    return avg_loss, accuracy, f1, precision, recall, all_preds, all_targets


def main():
    print("\n" + "="*70)
    print("LOADING DATASETS")
    print("="*70)

    datasets = {}

    try:
        X_train, y_train, X_test, y_test, activities = load_uci_har()
        datasets['UCI-HAR'] = {
            'X_train': X_train, 'y_train': y_train,
            'X_test': X_test, 'y_test': y_test,
            'activities': activities,
            'input_channels': X_train.shape[1],
            'seq_length': X_train.shape[2],
            'num_classes': len(np.unique(y_train))
        }
    except Exception as e:
        print(f"UCI-HAR failed: {e}")

    try:
        result = load_wisdm_data()
        if result[0] is not None:
            X_train, y_train, X_test, y_test, activities = result
            datasets['WISDM'] = {
                'X_train': X_train, 'y_train': y_train,
                'X_test': X_test, 'y_test': y_test,
                'activities': activities,
                'input_channels': X_train.shape[1],
                'seq_length': X_train.shape[2],
                'num_classes': len(np.unique(y_train))
            }
    except Exception as e:
        print(f"WISDM failed: {e}")

    try:
        result = load_pamap2_data()
        if result[0] is not None:
            X_train, y_train, X_test, y_test, activities = result
            datasets['PAMAP2'] = {
                'X_train': X_train, 'y_train': y_train,
                'X_test': X_test, 'y_test': y_test,
                'activities': activities,
                'input_channels': X_train.shape[1],
                'seq_length': X_train.shape[2],
                'num_classes': len(np.unique(y_train))
            }
    except Exception as e:
        print(f"PAMAP2 failed: {e}")

    try:
        result = load_mhealth_data()
        if result[0] is not None:
            X_train, y_train, X_test, y_test, activities = result
            datasets['MHEALTH'] = {
                'X_train': X_train, 'y_train': y_train,
                'X_test': X_test, 'y_test': y_test,
                'activities': activities,
                'input_channels': X_train.shape[1],
                'seq_length': X_train.shape[2],
                'num_classes': len(np.unique(y_train))
            }
    except Exception as e:
        print(f"MHEALTH failed: {e}")

    try:
        result = load_mobiact_data("/content/drive/MyDrive/HAR_Dataset/MOBIACT")
        if result[0] is not None:
            X_train, y_train, X_test, y_test, activities = result
            datasets['MobiAct'] = {
                'X_train': X_train, 'y_train': y_train,
                'X_test': X_test, 'y_test': y_test,
                'activities': activities,
                'input_channels': X_train.shape[1],
                'seq_length': X_train.shape[2],
                'num_classes': len(np.unique(y_train))
            }
    except Exception as e:
        print(f"MobiAct failed: {e}")

    try:
        result = load_motionsense_data()
        if result[0] is not None:
            X_train, y_train, X_test, y_test, activities = result
            datasets['MotionSense'] = {
                'X_train': X_train, 'y_train': y_train,
                'X_test': X_test, 'y_test': y_test,
                'activities': activities,
                'input_channels': X_train.shape[1],
                'seq_length': X_train.shape[2],
                'num_classes': len(np.unique(y_train))
            }
    except Exception as e:
        print(f"MotionSense failed: {e}")

    if not datasets:
        print("No datasets loaded! Exiting...")
        return

    print(f"\nLoaded {len(datasets)} datasets: {list(datasets.keys())}")

    all_results = []

    for dataset_name, data_info in datasets.items():
        print("\n" + "="*70)
        print(f"TRAINING ON: {dataset_name}")
        print("="*70)
        print(f"Train samples: {data_info['X_train'].shape[0]}")
        print(f"Test samples: {data_info['X_test'].shape[0]}")
        print(f"Input shape: ({data_info['input_channels']}, {data_info['seq_length']})")
        print(f"Classes: {data_info['num_classes']}")

        train_dataset = UCIHARDataset(data_info['X_train'], data_info['y_train'])
        test_dataset = UCIHARDataset(data_info['X_test'], data_info['y_test'])

        train_loader = DataLoader(
            train_dataset,
            batch_size=BATCH_SIZE,
            shuffle=True,
            num_workers=NUM_WORKERS,
            pin_memory=True,
            prefetch_factor=2
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=NUM_WORKERS,
            pin_memory=True,
            prefetch_factor=2
        )

        model = LightweightIMF(
            input_channels=data_info['input_channels'],
            hidden_dim=HIDDEN_DIM,
            latent_dim=LATENT_DIM,
            num_classes=data_info['num_classes'],
            seq_length=data_info['seq_length']
        ).to(device)

        print("\n" + "="*70)
        print("MODEL ANALYSIS")
        print("="*70)

        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Total Parameters: {total_params:,} ({total_params/1e6:.2f}M)")
        print(f"Trainable Parameters: {trainable_params:,} ({trainable_params/1e6:.2f}M)")

        flops_value = 0.0
        params_value = 0.0
        inf_time = 0.0

        if THOP_AVAILABLE:
            input_shape = (data_info['input_channels'], data_info['seq_length'])
            flops_value, params_value = compute_flops_params(model, input_shape, device)
            print(f"FLOPs: {flops_value:.2f}M")
            print(f"Params (thop): {params_value:.2f}M")

            inf_time = measure_inference_time(model, input_shape, device)
            print(f"Inference Time: {inf_time:.2f}ms")

        optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
        scaler = GradScaler('cuda', enabled=USE_AMP)

        print("\n" + "="*70)
        print("TRAINING STARTS")
        print("="*70)

        best_acc = 0
        best_f1 = 0

        for epoch in range(1, EPOCHS + 1):
            train_metrics = train_epoch(model, train_loader, optimizer, scaler, epoch, data_info['seq_length'])
            test_loss, test_acc, test_f1, test_prec, test_rec, _, _ = evaluate(model, test_loader)
            scheduler.step()

            if epoch % 20 == 0 or epoch == 1:
                print(f"\n{'='*70}")
                print(f"EPOCH {epoch}/{EPOCHS} | Dataset: {dataset_name}")
                print(f"{'='*70}")
                print(f"TRAIN - Loss: {train_metrics['loss']:.4f} | Acc: {train_metrics['acc']:.2f}%")
                print(f"   CE: {train_metrics['ce']:.4f} | Recon: {train_metrics['recon']:.4f} | Vel: {train_metrics['vel']:.4f} | Smooth: {train_metrics['smooth']:.4f}")
                print(f"TEST  - Loss: {test_loss:.4f} | Acc: {test_acc:.2f}% | F1: {test_f1:.2f}% | Prec: {test_prec:.2f}% | Rec: {test_rec:.2f}%")
                print(f"LR: {optimizer.param_groups[0]['lr']:.6f}")

                if torch.cuda.is_available():
                    print(f"GPU Memory: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

            if test_acc > best_acc:
                best_acc = test_acc
                best_f1 = test_f1
                torch.save(model.state_dict(), f'imf_lightweight_{dataset_name}.pth')
                if epoch % 20 == 0 or epoch == 1:
                    print(f"New best model saved! Acc: {best_acc:.2f}% | F1: {best_f1:.2f}%")

            if epoch % 50 == 0:
                torch.cuda.empty_cache()
                gc.collect()

        print("\n" + "="*70)
        print(f"FINAL RESULTS - {dataset_name}")
        print("="*70)

        model.load_state_dict(torch.load(f'imf_lightweight_{dataset_name}.pth'))
        _, final_acc, final_f1, final_prec, final_rec, preds, targets, latents = evaluate(model, test_loader, return_latents=True)

        cm = confusion_matrix(targets, preds)
        print("\nConfusion Matrix:")
        print(cm)

        plot_confusion_matrix(cm, data_info['activities'], dataset_name)
        print(f"Confusion matrix saved: confusion_matrix_{dataset_name}.png")

        plot_tsne(latents, np.array(targets), data_info['activities'], dataset_name)
        print(f"t-SNE plot saved: tsne_{dataset_name}.png")

        print("\nPer-class Accuracy:")
        for i, activity in enumerate(data_info['activities']):
            if i < len(cm):
                class_acc = cm[i, i] / cm[i].sum() * 100 if cm[i].sum() > 0 else 0
                print(f"  {activity}: {class_acc:.2f}%")

        print("\n" + "="*70)
        print(f"SUMMARY - {dataset_name}")
        print("="*70)
        print(f"Accuracy:  {final_acc:.2f}%")
        print(f"F1 Score:  {final_f1:.2f}%")
        print(f"Precision: {final_prec:.2f}%")
        print(f"Recall:    {final_rec:.2f}%")
        print(f"Params:    {total_params/1e6:.2f}M")
        print(f"FLOPs:     {flops_value:.2f}M")
        print(f"Inference: {inf_time:.2f}ms")
        print("="*70)

        all_results.append({
            'Dataset': dataset_name,
            'Accuracy': final_acc,
            'F1': final_f1,
            'Precision': final_prec,
            'Recall': final_rec,
            'Params (M)': total_params/1e6,
            'FLOPs (M)': flops_value,
            'Inference (ms)': inf_time
        })

        del model
        torch.cuda.empty_cache()
        gc.collect()

    print("\n" + "="*70)
    print("OVERALL RESULTS")
    print("="*70)

    results_df = pd.DataFrame(all_results)
    print("\n")
    print(results_df.to_string(index=False))

    try:
        from IPython.display import display
        display(results_df)
    except:
        pass

    results_df.to_csv('imf_results.csv', index=False)
    print("\nResults saved to: imf_results.csv")


if __name__ == '__main__':
    main()

Using device: cuda
GPU: NVIDIA A100-SXM4-40GB
GPU Memory: 39.56 GB
CONFIGURATION
Batch Size: 512 (Effective: 1024)
Gradient Accumulation Steps: 2
Epochs: 50
Learning Rate: 0.001
Hidden Dim: 128 | Latent Dim: 64
SIREN Layers: 3 | Frequencies: 16
Mixed Precision: True
Num Workers: 4

LOADING DATASETS

[UCI-HAR] Loading from: /content/drive/MyDrive/HAR_Dataset/UCI
  Loading: total_acc_x_train.txt
  Loading: total_acc_y_train.txt
  Loading: total_acc_z_train.txt
  Loading: body_acc_x_train.txt
  Loading: body_acc_y_train.txt
  Loading: body_acc_z_train.txt
  Loading: body_gyro_x_train.txt
  Loading: body_gyro_y_train.txt
  Loading: body_gyro_z_train.txt
  Loading: total_acc_x_test.txt
  Loading: total_acc_y_test.txt
  Loading: total_acc_z_test.txt
  Loading: body_acc_x_test.txt
  Loading: body_acc_y_test.txt
  Loading: body_acc_z_test.txt
  Loading: body_gyro_x_test.txt
  Loading: body_gyro_y_test.txt
  Loading: body_gyro_z_test.txt
  Train: (7352, 128, 9), Test: (2947, 128, 9)

[WISDM] Lo

,Dataset,Accuracy,F1,Precision,Recall,Params (M),FLOPs (M),Inference (ms)
0,UCI-HAR,87.818120,87.782749,87.891915,87.818120,0.282742,1.990912,1.479876
1,WISDM,99.055910,99.056377,99.060118,99.055910,0.102617,9.254912,1.554921
2,PAMAP2,95.752167,95.759665,95.783025,95.752167,0.125006,12.458240,1.550584
3,MHEALTH,99.732685,99.732615,99.733008,99.732685,0.129329,6.336128,1.535749
4,MobiAct,97.942122,97.942445,97.950920,97.942122,0.111650,11.918336,1.533730
5,MotionSense,96.119734,96.145825,96.333395,96.119734,0.115586,15.467264,1.534336



Results saved to: imf_results.csv
